In [1]:
# Load dataset
from torch_geometric.datasets import Planetoid
dataset = Planetoid("./", "Cora")
data = dataset[0]
print("Cora", data)

Cora Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [2]:
import torch

In [3]:
# Implementation of GCN from scratch
A = torch.eye(data.num_nodes) + torch.sparse.FloatTensor(data.edge_index, torch.ones(data.edge_index.shape[1]))
print(A.shape)
degree_hat = A.sum(dim=1).pow(-0.5)
D_inv_sqrt = torch.diag(degree_hat)
S = D_inv_sqrt.mm(A).mm(D_inv_sqrt)

/var/folders/_8/zs41_k7j63x2qgtj1z6fr2v00000gn/T/ipykernel_76917/1014638110.py:2: UserWarning: torch.sparse.SparseTensor(indices, values, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, dtype=, device=). (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1687072158581/work/torch/csrc/utils/tensor_new.cpp:595.)
  A = torch.eye(data.num_nodes) + torch.sparse.FloatTensor(data.edge_index, torch.ones(data.edge_index.shape[1]))


torch.Size([2708, 2708])
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [7]:
import torch.nn as nn

class GCN(nn.Module):

    def __init__(self, n_features, n_classes, hidden_size):
        super(GCN, self).__init__()
        self.linear = nn.Linear(n_features, hidden_size)
        self.leaky_relu = nn.LeakyReLU()
        self.out = nn.Linear(hidden_size, n_classes)
    

    def forward(self, x):
        x = self.linear(x)
        x = S.mm(x)
        x = self.leaky_relu(x)
        x = self.out(x)
        return x


In [8]:
import torch.optim as optim

In [11]:

model = GCN(data.x.shape[1], dataset.num_classes, 125)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [12]:
model.train()

num_epochs = 200
for epoch in range(num_epochs):
    optimizer.zero_grad()
    out = model(data.x)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

/Users/arindams_mac_m2_pro/opt/anaconda3/envs/py310/lib/python3.10/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


Epoch 0, Loss: 1.9501
Epoch 10, Loss: 0.0270
Epoch 20, Loss: 0.0007
Epoch 30, Loss: 0.0013
Epoch 40, Loss: 0.0065
Epoch 50, Loss: 0.0108
Epoch 60, Loss: 0.0087
Epoch 70, Loss: 0.0082
Epoch 80, Loss: 0.0082
Epoch 90, Loss: 0.0077
Epoch 100, Loss: 0.0074
Epoch 110, Loss: 0.0071
Epoch 120, Loss: 0.0068
Epoch 130, Loss: 0.0066
Epoch 140, Loss: 0.0064
Epoch 150, Loss: 0.0063
Epoch 160, Loss: 0.0061
Epoch 170, Loss: 0.0060
Epoch 180, Loss: 0.0059
Epoch 190, Loss: 0.0058


In [13]:
# Model performance
model.eval()
_, pred = out.max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print(f"Accuracy: {acc:.4f}")

Accuracy: 0.7820
